# Imports

## Installs

In [1]:

#!pip install layoutparser torchvision
#!pip install layoutparser[ocr]
#!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
#!apt-get update
#!apt-get install -y tesseract-ocr
#!pip install Pillow==9.5.0

#!pip install pytesseract


!pip install transformers

!pip install -q --upgrade transformers accelerate optimum
!pip install -q --no-build-isolation auto-gptq



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Imports



In [115]:
import pandas as pd
import numpy as np
import time
import tqdm
from tqdm import tqdm
import json
import torch
print(torch.__version__)  # Should print a compatible version
print(torch.cuda.is_available())  #


import glob
import os
import re
import random
import cv2
import PIL
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline



from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification, AutoModelForCausalLM
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import pipeline
from huggingface_hub import login

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import accelerate 
from accelerate import Accelerator


import warnings
warnings.filterwarnings('ignore')


2.5.1+cu124
True


In [152]:


os.environ['HOME'] = '/scratch/$USER'  # Adjust this path if necessary
os.environ['PYTHONUSERBASE'] = '/scratch/$USER/.local'

# Set the PIP_CACHE_DIR environment variable
os.environ['PIP_CACHE_DIR'] = '/scratch/$USER/pip_cache'
os.environ['HF_HOME'] = '/scratch/$USER/pip_cache'


## Loading Data

In [9]:
df = pd.read_csv("../Data/news_master.csv")

In [10]:
df

,Date,arabnews,cnbc,ei,ft,mees,oilprice,opec
0,2021-01-04,WEEKLY ENERGY RECAP: Oil prices on an upward m...,Shares of Chinese telecom giants surge followi...,Schlumberger Optimistic for 2021 Recovery,Saudi Arabia warns oil producers over fragile ...,Opec Aims For More Stable 2021. US Mideast Cru...,Oil Rises As Market Awaits OPEC+ Production De...,The 13th OPEC and non-OPEC Ministerial Meeting...
1,2021-01-05,WEEKLY ENERGY RECAP: Oil prices on an upward m...,Oil prices will remain flat until mid-2021: JB...,Schlumberger Optimistic for 2021 Recovery,Saudis pledge to cut oil output despite Russia...,Opec Aims For More Stable 2021. US Mideast Cru...,OPEC+ Meeting Ends With Major Surprise Cut Fro...,13th OPEC and non-OPEC Ministerial Meeting con...
2,2021-01-06,WEEKLY ENERGY RECAP: Oil prices on an upward m...,Oil rises as supply constraints retain focus a...,Schlumberger Optimistic for 2021 Recovery,Oil auction in Arctic wildlife refuge draws sc...,Opec Aims For More Stable 2021. US Mideast Cru...,OPEC+ Meeting Ends With Major Surprise Cut Fro...,13th OPEC and non-OPEC Ministerial Meeting con...
3,2021-01-07,WEEKLY ENERGY RECAP: Oil prices on an upward m...,"Oil hits highest level since February, posts n...",Schlumberger Optimistic for 2021 Recovery,A Democratic Senate provides a surge for Biden...,Opec Aims For More Stable 2021. US Mideast Cru...,The Next 5 Days Could See A Buying Spree In Oi...,13th OPEC and non-OPEC Ministerial Meeting con...
4,2021-01-08,WEEKLY ENERGY RECAP: Oil prices on an upward m...,"Oil hits highest level since February, posts n...",Schlumberger Optimistic for 2021 Recovery,UK’s FTSE 100 kicks off 2021 with 6% rally lif...,Algeria’s Gas Exports Slump To 25-Year Low For...,Oil Rally Continues Despite Slow Vaccine Rollout,13th OPEC and non-OPEC Ministerial Meeting con...
...,...,...,...,...,...,...,...,...
1371,2024-10-06,Oman credit rating rises to ‘BBB-’ from ‘BB+’:...,Oil pares gains after strongest weekly rise in...,US Rig Count Enters Fourth Quarter in Decline....,TotalEnergies considers foray into copper trad...,Compliance In Spotlight At Opec JMMC Meeting,Oil Explodes 4% Amid Talk of Israel Attacking ...,56th Meeting of the Joint Ministerial Monitori...
1372,2024-10-07,Oman credit rating rises to ‘BBB-’ from ‘BB+’:...,Oil edges lower after rallying to highest in o...,Aramco Hikes November Prices for Asia Crude Ex...,Oil surges above $80 as conflict and storm spa...,Compliance In Spotlight At Opec JMMC Meeting,Historic Short Squeeze Sends Oil Prices Higher,56th Meeting of the Joint Ministerial Monitori...
1373,2024-10-08,"Pakistan’s stock market surpasses historic 85,...",Oil prices steady after sliding on potential I...,Demand Concerns Outweigh Mideast Turmoil in EI...,Chevron in talks to sell Texas gas assets to T...,Compliance In Spotlight At Opec JMMC Meeting,Historic Short Squeeze Sends Oil Prices Higher,56th Meeting of the Joint Ministerial Monitori...
1374,2024-10-09,"Pakistan’s stock market surpasses historic 85,...",Oil prices rise more than 3% on Israel-Iran co...,"Oil Prices Dip Amid Demand, Risk Crosscurrents...",Who and what is driving oil price volatility. ...,Compliance In Spotlight At Opec JMMC Meeting,Wild Oil Price Forecasts: Some Predict $350 if...,56th Meeting of the Joint Ministerial Monitori...


# Model Download and Initializaton

## Login Token

In [2]:
login(token = 'hidden')

## Llama 8b Instruct

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.device

In [9]:
model.device

device(type='cuda', index=0)

## Mistral NeMo

In [5]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)



Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
model.device

device(type='cuda', index=0)

## Twitter BERT


In [3]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"


config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, device_map = 'auto')





Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:

pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)



In [6]:
model.device

device(type='cuda', index=0)

## Prompts

### Llama Prompting

In [7]:
# Define extraction function for borrowers
def llama_sentiment(text):
    # Simplified prompt for extracting only borrowers
    prompt = f"""

      <|begin_of_text|><|start_header_id|>system<|end_header_id|>
      
      You are an oil and OPEC annoucement sentiment analyzer. You will be given a document with oil and OPEC news. You have 
      to give a sentiment score to that document based on what it means for the oil market. 

      Give a continuous score between -1.0 and 1.0 Give it up to one decimal place. 
      
        ** -1 being the worst news for oil price and sales. 
        ** -1 to 0 being bad news for oil price and sales. 
        ** 0 being neutral news for oil price and sales. 
        ** 0 to 1 being good news for oil price and sales. 
        ** 1 being exremely good news for oil price and sales. 


      Please present your output as a float. 

      Output Example for your reference:
      
      JSON: {{"sentiment_score": "0.2"}}
      
      <|eot_id|><|start_header_id|>user<|end_header_id|>

        
      [INST]
      This is the OPEC annoucement you have to analyze and output the sentiment for. 

      {text} <|eot_id|>

      <|start_header_id|>assistant<|end_header_id|>

      JSON: 
      
    """

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    inputs = inputs.input_ids
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Use max_new_tokens to control the output
    outputs = model.generate(inputs, max_new_tokens=512, temperature=0.2)
    extracted_info = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    # Clean up the string by removing unnecessary escape characters
    #cleaned_output = extracted_info.replace('\\n', '').replace('\\\'', '\'').replace('\'', '"')

    marker = "JSON:"
    start = extracted_info.rfind(marker)
    if start != -1:
        json_output = extracted_info[start + len(marker):].strip() 
        try: 
            extracted_json = json.loads(json_output)
            return extracted_json
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed: {e}. Raw JSON output was: {json_output}")
            return json_output
            
        
    else:
        return None
    

  

### Mistral Prompting

In [7]:
# Define extraction function for borrowers
def mistral_sentiment(text):
    # Simplified prompt for extracting only borrowers
    prompt = f"""

      <s>[INST]
      
      You are an oil and oil news sentiment analyzer. You will be given news headlines with oil and OPEC news. You have 
      to give a sentiment score to that document based on what it means for the oil market. 

      Give a continuous score between -1.0 and 1.0 Give it up to one decimal place. 
      
        ** -1 being the worst news for oil price and sales. 
        ** -1 to 0 being bad news for oil price and sales. 
        ** 0 being neutral news for oil price and sales. 
        ** 0 to 1 being good news for oil price and sales. 
        ** 1 being exremely good news for oil price and sales. 

      Please present your output as a float with one decimal point. 

      Output Example for your reference: 
      
      JSON: 
      
      {{"sentiment_score": "0.2"}}
      
      [/INST] </s>

        
      [INST]
      
      This is the news headline you have to analyze and output the sentiment for. 

      {text} [/INST]

      [INST]

      Now give a sentiment score. Please return only the JSON object. Do not include any special characters in the output like "\\n" or "\\t".  Start output after saying "JSON:" </s>

      [/INST]

      
      
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
 
    attention_mask = inputs.attention_mask
    inputs = inputs.input_ids

    # Use max_new_tokens to control the output
    outputs = model.generate(
        inputs, 
        attention_mask=attention_mask, 
        max_new_tokens=50, 
        temperature=0.2
        
        # Uses EOS token as pad token
    )
    #outputs = model.generate(inputs, max_new_tokens=512, temperature=0.1)
    extracted_info = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    # Clean up the string by removing unnecessary escape characters
    #cleaned_output = extracted_info.replace('\\n', '').replace('\\\'', '\'').replace('\'', '"')

    marker = "JSON:"
    start = extracted_info.rfind(marker)
    if start != -1:
        json_output = extracted_info[start + len(marker):].strip() #finding the JSON post "JSON:"
        try: 
            extracted_json = json.loads(json_output) #json.loads takes the json string output and converts that into a python object 
            return extracted_json
        except json.JSONDecodeError as e: #error
            try: 
                json_output_cleaned= json_output.strip().replace('\n', '').replace('\t', '').strip('"\\"').replace(r'\n', '').replace(r'\r', '').replace(r'\t', '').strip() #cleaning the output
                extracted_json = json.loads(json_output_cleaned)
                return extracted_json
            except: 
                print(f"JSON decoding failed: {e}. Raw JSON output was: {json_output}")
                return json_output
            
        
    else:
        return None
    

  

In [51]:

#     attention_mask = inputs['attention_mask']

#      #Generate with attention_mask and pad_token_id
#     outputs = model.generate(
#         input_ids, 
#         attention_mask=attention_mask, 
#         max_new_tokens=512, 
#         temperature=0.2,
#         pad_token_id=tokenizer.pad_token_id  # Uses EOS token as pad token
#     )

### BERT Prompting

In [108]:
def sentiment_bert(text): 

    output = pipe(text)
    output = output[0]

    
    if output['label'] == 'negative': 
        output['score'] = output['score']*-1
    elif output['label'] == 'neutral': 
         output['score'] = 0
        
    del output['label']

    
    return output



# Runs

## Manual Testing

In [129]:
text1 = df['opec'][100]


In [130]:
text1

'15th OPEC and non-OPEC Ministerial Meeting concludes\nNo 10/2021\nVienna, Austria\n01 Apr 2021\nThe 15th Meeting of OPEC and non-OPEC Ministers took place via videoconference on Thursday, 1 April 2021, under the Chairmanship of HRH Prince Abdul Aziz bin Salman, Saudi Arabia’s Minister of Energy, and Co-Chair HE Alexander Novak, Deputy Prime Minister of the Russian Federation.\nThe Meeting emphasized the ongoing positive contributions of the Declaration of Cooperation (DoC) in supporting a rebalancing of the global oil market. These outcomes are in line with the historic decisions taken at the 10th (Extraordinary) OPEC and non-OPEC Ministerial Meeting on 12 April 2020 to adjust downwards overall crude oil production, and subsequent decisions.\nThe Ministers noted, with gratitude, the value of the prudent policy approach by Saudi Arabia of maintaining its additional voluntary adjustments of 1 mb/d in April 2021 for the third month in a row.\nThe Meeting approved the adjustment of the pr

In [131]:
out = pipe(text1)

RuntimeError: The expanded size of the tensor (814) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 814].  Tensor sizes: [1, 514]

In [126]:
out 

[{'label': 'neutral', 'score': 0.9405133724212646}]

In [127]:
out = sentiment_bert(text1)

In [128]:
out

{'score': 0}

In [54]:
out[0]['label'] =='positive'

True

In [40]:
del out['label']

In [41]:
out

{'score': 0.9112440943717957}

## Automated

In [15]:
df.columns

Index(['Date', 'arabnews', 'cnbc', 'ei', 'ft', 'mees', 'oilprice', 'opec'], dtype='object')

In [24]:
for col in df.columns: 
    print(col)

Date
arabnews
cnbc
ei
ft
mees
oilprice
opec


In [117]:
sentiment_output_bert = {} #dict with all news sources as keys and their sentiment dict lists as values 

for col in tqdm(df.columns[1:], desc = 'News Source'):
    print(f'Getting Sentiment for {col}')
    source_sentiments = [] #sentiment dict list for each news source 
    for headline in tqdm(df[col], desc = 'Headline', bar_format='[{elapsed}<{remaining}] {n_fmt}/{total_fmt} | {l_bar}{bar} {rate_fmt}{postfix}', colour='yellow'):
        sentiment = sentiment_bert(headline)
        source_sentiments.append(sentiment)
    sentiment_output_bert[col] = source_sentiments



News Source:   0%|          | 0/7 [00:00<?, ?it/s]

Getting Sentiment for arabnews



[00:00<?] 0/1376 | Headline:   0%|           ?it/s
[00:00<00:11] 12/1376 | Headline:   1%|           118.63it/s
[00:00<00:11] 25/1376 | Headline:   2%|▏          119.87it/s
[00:00<00:11] 38/1376 | Headline:   3%|▎          120.34it/s
[00:00<00:10] 51/1376 | Headline:   4%|▎          120.67it/s
[00:00<00:10] 64/1376 | Headline:   5%|▍          121.15it/s
[00:00<00:10] 77/1376 | Headline:   6%|▌          121.64it/s
[00:00<00:10] 90/1376 | Headline:   7%|▋          122.38it/s
[00:00<00:10] 103/1376 | Headline:   7%|▋          121.71it/s
[00:00<00:10] 116/1376 | Headline:   8%|▊          121.38it/s
[00:01<00:10] 129/1376 | Headline:   9%|▉          122.31it/s
[00:01<00:10] 142/1376 | Headline:  10%|█          123.01it/s
[00:01<00:09] 155/1376 | Headline:  11%|█▏         123.13it/s
[00:01<00:09] 168/1376 | Headline:  12%|█▏         122.74it/s
[00:01<00:09] 181/1376 | Headline:  13%|█▎         122.84it/s
[00:01<00:09] 194/1376 | Headline:  14%|█▍         123.08it/s
[00:01<00:09] 207/1376 | 

Getting Sentiment for cnbc



[00:00<?] 0/1376 | Headline:   0%|           ?it/s
[00:00<00:11] 12/1376 | Headline:   1%|           116.85it/s
[00:00<00:11] 24/1376 | Headline:   2%|▏          117.58it/s
[00:00<00:11] 36/1376 | Headline:   3%|▎          117.11it/s
[00:00<00:11] 48/1376 | Headline:   3%|▎          117.25it/s
[00:00<00:11] 60/1376 | Headline:   4%|▍          117.03it/s
[00:00<00:11] 72/1376 | Headline:   5%|▌          117.50it/s
[00:00<00:11] 84/1376 | Headline:   6%|▌          116.92it/s
[00:00<00:10] 96/1376 | Headline:   7%|▋          117.25it/s
[00:00<00:10] 108/1376 | Headline:   8%|▊          117.64it/s
[00:01<00:10] 120/1376 | Headline:   9%|▊          118.05it/s
[00:01<00:10] 132/1376 | Headline:  10%|▉          118.05it/s
[00:01<00:10] 144/1376 | Headline:  10%|█          118.29it/s
[00:01<00:10] 156/1376 | Headline:  11%|█▏         118.02it/s
[00:01<00:10] 168/1376 | Headline:  12%|█▏         118.04it/s
[00:01<00:10] 180/1376 | Headline:  13%|█▎         117.47it/s
[00:01<00:10] 192/1376 | H

...

Getting Sentiment for oilprice



[00:00<?] 0/1376 | Headline:   0%|           ?it/s
[00:00<00:13] 11/1376 | Headline:   1%|           101.33it/s
[00:00<00:13] 22/1376 | Headline:   2%|▏          100.42it/s
[00:00<00:13] 33/1376 | Headline:   2%|▏          100.42it/s
[00:00<00:13] 44/1376 | Headline:   3%|▎          100.48it/s
[00:00<00:13] 55/1376 | Headline:   4%|▍          100.38it/s
[00:00<00:13] 66/1376 | Headline:   5%|▍          100.65it/s
[00:00<00:12] 77/1376 | Headline:   6%|▌          101.14it/s
[00:00<00:12] 88/1376 | Headline:   6%|▋          101.21it/s
[00:00<00:12] 99/1376 | Headline:   7%|▋          101.03it/s
[00:01<00:12] 110/1376 | Headline:   8%|▊          100.87it/s
[00:01<00:12] 121/1376 | Headline:   9%|▉          100.73it/s
[00:01<00:12] 132/1376 | Headline:  10%|▉          100.32it/s
[00:01<00:12] 143/1376 | Headline:  10%|█          100.47it/s
[00:01<00:12] 154/1376 | Headline:  11%|█          100.40it/s
[00:01<00:12] 165/1376 | Headline:  12%|█▏         100.66it/s
[00:01<00:11] 176/1376 | He

Getting Sentiment for opec



[00:00<17:27] 1/1376 | Headline:   0%|            1.31it/s
News Source:  86%|████████▌ | 6/7 [01:12<00:12, 12.00s/it]


RuntimeError: The expanded size of the tensor (845) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 845].  Tensor sizes: [1, 514]

In [121]:
opec = [sentiment_bert(headline) for headline in df['opec']]

RuntimeError: The expanded size of the tensor (845) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 845].  Tensor sizes: [1, 514]

In [132]:
sentiment_output_bert.keys()


dict_keys(['arabnews', 'cnbc', 'ei', 'ft', 'mees', 'oilprice'])

In [149]:
oilprice_df = pd.DataFrame(sentiment_output_bert['oilprice'])
oilprice_df

,score
0,0.000000
1,-0.526396
2,-0.526396
3,0.000000
4,0.711697
...,...
1371,-0.788622
1372,0.000000
1373,0.000000
1374,0.000000


In [151]:
oilprice_df.to_csv('../Outputs/oilprice_bert.csv' )

In [16]:
df_mistral = pd.DataFrame(sentiments)
df_mistral

,sentiment_score
0,0.0
1,0.5
2,0.5
3,0.4
4,0.5
...,...
127,0.5
128,-0.5
129,0.0
130,0.5


# Outputs

## Old

In [78]:
#df['sentiment_mistral'] = sentiment_2_df['sentiment_score']

,date,text,sentiment_mistral
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5
...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5


In [80]:
os.getcwd()

'/scratch/us2055/Oil_Price/Notebooks'

In [81]:
df.to_csv('../Outputs/sentiment_mistral.csv')

## Appending Sentiment

In [21]:
df_sentiment_combined = pd.read_csv("../Outputs/sentiment_combined.csv")
df_sentiment_combined 

,Unnamed: 0.1,Unnamed: 0,date,text,sentiment_mistral,sentiment_llama
0,0,0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1
1,1,1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0
2,2,2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0
3,3,3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0
4,4,4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0
...,...,...,...,...,...,...
127,127,127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5
128,128,128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6
129,129,129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5
130,130,130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0


In [25]:
df_sentiment_combined['sentiment_mistral_1'] = df_mistral['sentiment_score']
df_sentiment_combined = df_sentiment_combined.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
df_sentiment_combined

,date,text,sentiment_mistral,sentiment_llama,sentiment_mistral_1
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0,0.5
...,...,...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6,-0.5
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5,0.0
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0,0.5


In [28]:
df_sentiment_combined.to_csv('../Outputs/sentiment_combined_1.csv', index = False)

In [29]:
df_sentiment_combined

,date,text,sentiment_mistral,sentiment_llama,sentiment_mistral_1
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0,0.5
...,...,...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6,-0.5
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5,0.0
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0,0.5


## New

In [156]:
arabnews_mistral = pd.read_csv('../Outputs/arabnews_mistral.csv')
cnbc_mistral = pd.read_csv('../Outputs/cnbc_mistral.csv')
ei_mistral = pd.read_csv('../Outputs/ei_mistral.csv')
ft_mistral = pd.read_csv('../Outputs/ft_mistral.csv')
mees_mistral = pd.read_csv('../Outputs/mees_mistral.csv')
oilprice_mistral = pd.read_csv('../Outputs/oilprice_mistral.csv')
opec_mistral = pd.read_csv('../Outputs/opec_mistral.csv')                           

In [158]:
arabnews_bert = pd.read_csv('../Outputs/arabnews_bert.csv')
cnbc_bert = pd.read_csv('../Outputs/cnbc_bert.csv')
ei_bert = pd.read_csv('../Outputs/ei_bert.csv')
ft_bert = pd.read_csv('../Outputs/ft_bert.csv')
mees_bert = pd.read_csv('../Outputs/mees_bert.csv')
oilprice_bert = pd.read_csv('../Outputs/oilprice_bert.csv')



In [172]:
mistral_df = pd.DataFrame(zip(df['Date'], 
                              arabnews_mistral['sentiment_score'], 
                              cnbc_mistral['sentiment_score'], 
                              ei_mistral['sentiment_score'], 
                              ft_mistral['sentiment_score'], 
                              mees_mistral['sentiment_score'], 
                              oilprice_mistral['sentiment_score'], 
                              opec_mistral['sentiment_score']), 
                           columns=['Date', 'arabnews', 'cnbc', 'ei', 'ft', 'mees', 'oilprice', 'opec'])

In [173]:
mistral_df

,Date,arabnews,cnbc,ei,ft,mees,oilprice,opec
0,2021-01-04,0.4,0.0,0.5,-0.5,0.4,0.4,0.0
1,2021-01-05,0.4,0.4,0.5,0.5,0.4,0.7,0.4
2,2021-01-06,0.4,0.4,0.5,-0.5,0.4,0.7,0.4
3,2021-01-07,0.4,0.7,0.5,0.5,0.4,0.5,0.4
4,2021-01-08,0.4,0.7,0.5,0.4,-0.5,0.5,0.4
...,...,...,...,...,...,...,...,...
1371,2024-10-06,0.0,0.0,0.4,0.0,-0.3,0.6,0.4
1372,2024-10-07,0.0,0.5,0.6,0.6,-0.3,0.7,0.4
1373,2024-10-08,0.0,-0.3,-0.3,-0.5,-0.3,0.7,0.4
1374,2024-10-09,0.0,0.3,-0.3,0.4,-0.3,-0.5,0.4


In [181]:
mistral_df.to_csv('../Outputs/mistral_combined.csv', index = False)

In [178]:
bert_df = pd.DataFrame(zip(df['Date'], 
                           arabnews_bert['score'], 
                           cnbc_bert['score'], 
                           ei_bert['score'], 
                           ft_bert['score'], 
                           mees_bert['score'], 
                           oilprice_bert['score']), 
                       columns=['Date', 'arabnews', 'cnbc', 'ei', 'ft', 'mees', 'oilprice'])


In [179]:
bert_df

,Date,arabnews,cnbc,ei,ft,mees,oilprice
0,2021-01-04,0.519839,0.000000,0.889924,-0.569241,0.000000,0.000000
1,2021-01-05,0.519839,0.000000,0.889924,-0.560321,0.000000,-0.526396
2,2021-01-06,0.519839,0.000000,0.889924,-0.654790,0.000000,-0.526396
3,2021-01-07,0.519839,0.848916,0.889924,0.644426,0.000000,0.000000
4,2021-01-08,0.519839,0.848916,0.889924,0.764327,-0.626426,0.711697
...,...,...,...,...,...,...,...
1371,2024-10-06,0.000000,0.652695,0.000000,0.000000,0.000000,-0.788622
1372,2024-10-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1373,2024-10-08,0.586104,0.000000,-0.603633,-0.496074,0.000000,0.000000
1374,2024-10-09,0.586104,0.000000,0.000000,0.000000,0.000000,0.000000


In [180]:
bert_df.to_csv('../Outputs/bert_combined.csv', index = False)